
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


In [98]:
import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np

import pickle

In [99]:
def relabel_axes(old_min, old_max, new_min, new_max):
    scale = (new_max - new_min) / (old_max - old_min)

    def format_func(value, tick_number):
        return new_min + value * scale

    plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))

### grab the data and set up variables

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)



print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
for sub in subjects:

    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data

    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


    # do filtering now
    ## Crop raw data to minimize processing time
    new = crop_empty_data(filt, )

    # Mark channel outliers as bad
    # new.info['bads'] = channel_outlier_marker(new, 4)
    # Exclude bad channels
    good = new.copy().drop_channels(filt.info['bads'])
    good.load_data()

    # CAR
    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Remove intermediates from mem
    del new

    # make baseline 
    # make stimulus baseline EpochsTFR
    times=[-1, 0.5]

    trials = trial_ieeg(good, "Stimulus", times, preload=True)
    # outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    data_dict = {}  # Create an empty dictionary

    for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, event, times, preload=True)
        # outliers_to_nan(trials, outliers=10)
        HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
        crop_pad(HG_ev1, "0.5s")

        HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore') #removed .average() part. Check with Aaron if this is okay.
        # HG_ev1.resample(100)
        # HG_ev1.filenames = good.filenames

        # Store the variable in the dictionary with the desired name
        data_dict[f"HG_ev1_{event}"] = HG_ev1
        data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled
        


    resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
    resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

    # # # Manually interpolate NaN values
    # resp_evoke.data = np.where(np.isnan(resp_evoke.data), np.interp(resp_evoke.times, resp_evoke.times[~np.isnan(resp_evoke.data[0])], resp_evoke.data[0, ~np.isnan(resp_evoke.data[0])]), resp_evoke.data)

    # # Exclude time points with NaN values
    # valid_time_points = ~np.isnan(resp_evoke.data[0])  # Filter time points without NaNs
    # resp_evoke = resp_evoke.crop(tmin=resp_evoke.times[valid_time_points][0], tmax=resp_evoke.times[valid_time_points][-1])

    # Plot the evoked data
    fig = resp_evoke_rescaled.plot()
    fig.savefig(save_dir + '_HG_ev1_Response_rescaled_zscore.png')

    fig = stim_evoke_rescaled.plot()
    fig.savefig(save_dir + '_HG_ev1_Stimulus_rescaled_zscore.png')


    # # decimate for stats
    # resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
    # stim = data_dict["HG_ev1_Stimulus"]
    # HG_base.decimate(2)
    # resp.decimate(2)
    # stim.decimate(2)

    # # import scipy
    # resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
    #                             n_perm=1000, n_jobs=6, ignore_adjacency=1)
    # stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
    #                             n_perm=1000, n_jobs=6, ignore_adjacency=1)
    



    # # plot stats
    # plt.figure(figsize=(16, 12))  # Adjust the width and height as needed
    # plt.imshow(stim_mask)
    # # hold on from matlab
    # # plt.imshow(x) #plot multiple things like this

    # relabel_axes(0, 2500, -1000, 1500)
    # plt.savefig(save_dir + '_stimulus_stats.png', dpi=300)

    # plt.figure(figsize=(16, 12))
    # plt.imshow(resp_mask)
    # relabel_axes(0, 2500, -1000, 1500)
    # plt.savefig(save_dir + '_response_stats.png', dpi=300)


### for testing with a single subject

In [111]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)



print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
sub = 'D0057'

filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False) #get line-noise filtered data

save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


# do filtering now
## Crop raw data to minimize processing time
new = crop_empty_data(filt, )

# Mark channel outliers as bad
# new.info['bads'] = channel_outlier_marker(new, 4)
# Exclude bad channels
good = new.copy().drop_channels(filt.info['bads'])
good.load_data()

# CAR
ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# Remove intermediates from mem
del new

# make baseline 
# make stimulus baseline EpochsTFR
times=[-1, 0.5]

trials = trial_ieeg(good, "Stimulus", times, preload=True)
# outliers_to_nan(trials, outliers=10)
HG_base = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base, "0.5s")

data_dict = {}  # Create an empty dictionary

for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
    times = [None, None]
    times[0] = t[0] - 0.5
    times[1] = t[1] + 0.5
    trials = trial_ieeg(good, event, times, preload=True)
    # outliers_to_nan(trials, outliers=10)
    HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_ev1, "0.5s")

    HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore') #removed .average() part. Check with Aaron if this is okay.
    # HG_ev1.resample(100)
    # HG_ev1.filenames = good.filenames

    # Store the variable in the dictionary with the desired name
    data_dict[f"HG_ev1_{event}"] = HG_ev1
    data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled
    


resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))


# this is broken cuz it's trying to use the .average method from mne that has shape requirements
# resp_evoke_channel_avg = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))
# resp_evoke_rescaled_channel_avg = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))
# stim_evoke_channel_avg = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))
# stim_evoke_rescaled_channel_avg = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=(0, 1)))

# # # Manually interpolate NaN values
# resp_evoke.data = np.where(np.isnan(resp_evoke.data), np.interp(resp_evoke.times, resp_evoke.times[~np.isnan(resp_evoke.data[0])], resp_evoke.data[0, ~np.isnan(resp_evoke.data[0])]), resp_evoke.data)

# # Exclude time points with NaN values
# valid_time_points = ~np.isnan(resp_evoke.data[0])  # Filter time points without NaNs
# resp_evoke = resp_evoke.crop(tmin=resp_evoke.times[valid_time_points][0], tmax=resp_evoke.times[valid_time_points][-1])

# Plot the evoked data
fig = resp_evoke_rescaled.plot()
fig.add_channels([resp_evoke_rescaled_channel_avg])

fig.savefig(save_dir + '_HG_ev1_Response_rescaled_zscore_test.png')

fig = stim_evoke_rescaled.plot()
fig.add_channels([stim_evoke_rescaled_channel_avg])

fig.savefig(save_dir + '_HG_ev1_Stimulus_rescaled_zscore_test.png')


# # decimate for stats
# resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
# stim = data_dict["HG_ev1_Stimulus"]
# HG_base.decimate(2)
# resp.decimate(2)
# stim.decimate(2)

# # import scipy
# resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
#                             n_perm=1000, n_jobs=6, ignore_adjacency=1)
# stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
#                             n_perm=1000, n_jobs=6, ignore_adjacency=1)




# # plot stats
# plt.figure(figsize=(16, 12))  # Adjust the width and height as needed
# plt.imshow(stim_mask)
# # hold on from matlab
# # plt.imshow(x) #plot multiple things like this

# relabel_axes(0, 2500, -1000, 1500)
# plt.savefig(save_dir + '_stimulus_stats.png', dpi=300)

# plt.figure(figsize=(16, 12))
# plt.imshow(resp_mask)
# relabel_axes(0, 2500, -1000, 1500)
# plt.savefig(save_dir + '_response_stats.png', dpi=300)


{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24})
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.t

c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


...

100%|██████████| 448/448 [09:07<00:00,  1.22s/it]


Applying baseline correction (mode: zscore)
Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/n75', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimulus/i75/r25', 'Stimulus/i75/r75', 'Stimulus/i75/s25', 'Stimulus/i75/s75']
event_ids: {'c25/n25': 1, 'c25/r25': 2, 'c25/r75': 3, 'c25/s25': 4, 'c25/s75': 5, 'c75/r25': 6, 'c75/r75': 7, 'c75/s25': 8, 'c75/s75': 9, 'i25/r

100%|██████████| 436/436 [09:01<00:00,  1.24s/it]


Applying baseline correction (mode: zscore)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


RuntimeError: You passed a function that resulted n data of shape (5121,), but it should be (178, 5121).

In [112]:
data = HG_ev1._data
base_data = HG_base._data
rescaled_data = HG_ev1_rescaled._data

In [116]:
axis: tuple[int] | int = -1
mean = np.nanmean(HG_base, axis=axis, keepdims=True)
std = np.nanstd(HG_base, axis=axis, keepdims=True)

In [129]:
copy = HG_ev1.copy()
copy /= mean

ValueError: operands could not be broadcast together with shapes (436,178,5121) (448,178,1) 

In [127]:
test = rescale(HG_ev1, HG_base, mode='ratio', copy=True)

Applying baseline correction (mode: ratio)


In [126]:
HG_ev1._data

array([[[3.52057441e-06, 3.53875816e-06, 3.55698057e-06, ...,
         5.48328535e-06, 5.40768906e-06, 5.33345019e-06],
        [3.29034856e-06, 3.29179898e-06, 3.29309478e-06, ...,
         6.75980755e-06, 6.69208748e-06, 6.62515777e-06],
        [3.44067757e-06, 3.43543979e-06, 3.42967087e-06, ...,
         7.65957247e-06, 7.55610017e-06, 7.45419129e-06],
        ...,
        [4.22477069e-06, 4.19639036e-06, 4.16786133e-06, ...,
         9.32454714e-06, 9.18627848e-06, 9.04900662e-06],
        [5.35472645e-06, 5.36452353e-06, 5.37396136e-06, ...,
         8.94728691e-06, 8.81102187e-06, 8.67616927e-06],
        [5.15685406e-06, 5.16389355e-06, 5.17066383e-06, ...,
         9.34872696e-06, 9.21001447e-06, 9.07250615e-06]],

       [[5.39125858e-06, 5.39561461e-06, 5.39969187e-06, ...,
         3.98601424e-06, 3.99246028e-06, 3.99956798e-06],
        [5.95640358e-06, 5.95050096e-06, 5.94379253e-06, ...,
         3.30350736e-06, 3.30122430e-06, 3.29863587e-06],
        [5.95745723e-06, 

In [124]:
test._data

array([[[-1.36217881, -1.36007913, -1.35797497, ..., -1.13554354,
         -1.14427268, -1.15284508],
        [-1.3454796 , -1.34531334, -1.34516481, ..., -0.94777586,
         -0.9555386 , -0.96321075],
        [-1.33172459, -1.33232633, -1.33298909, ..., -0.84703873,
         -0.8589261 , -0.87063385],
        ...,
        [-1.33020204, -1.33352967, -1.33687472, ..., -0.73224781,
         -0.74845996, -0.76455523],
        [-1.20252869, -1.20139227, -1.20029752, ..., -0.78580711,
         -0.80161327, -0.81725559],
        [-1.28651427, -1.28569803, -1.284913  , ..., -0.80045907,
         -0.81654304, -0.83248737]],

       [[-1.14616992, -1.14566693, -1.14519612, ..., -1.30843423,
         -1.3076899 , -1.30686917],
        [-1.03986998, -1.04054659, -1.04131558, ..., -1.34397121,
         -1.34423292, -1.34452963],
        [-1.04258548, -1.040737  , -1.03892755, ..., -1.27100757,
         -1.27196164, -1.27298791],
        ...,
        [-1.27974617, -1.27899309, -1.27825264, ..., -

In [119]:
def fun(d, m, s):
    d -= m
    d /= s

test = fun(HG_ev1, mean, std)

ValueError: operands could not be broadcast together with shapes (436,178,5121) (448,178,1) 

In [104]:
axis

-1

In [94]:
base_data

array([[[8.04202136e-06, 8.06630487e-06, 8.09128869e-06, ...,
         1.06568850e-05, 1.06555872e-05, 1.06534771e-05],
        [7.58790247e-06, 7.59244813e-06, 7.59625345e-06, ...,
         9.29539419e-06, 9.30285933e-06, 9.30969509e-06],
        [5.16313230e-06, 5.15927695e-06, 5.15480951e-06, ...,
         1.06597072e-05, 1.06316638e-05, 1.06029320e-05],
        ...,
        [7.00386136e-06, 7.02139732e-06, 7.03766455e-06, ...,
         5.74739215e-06, 5.75310969e-06, 5.75986815e-06],
        [5.50127061e-06, 5.50241384e-06, 5.50287223e-06, ...,
         5.31490787e-06, 5.31508795e-06, 5.31588830e-06],
        [4.74779608e-06, 4.75412344e-06, 4.75971228e-06, ...,
         5.32444619e-06, 5.33681941e-06, 5.34966239e-06]],

       [[6.54741734e-06, 6.50222000e-06, 6.45706268e-06, ...,
         9.01476051e-06, 8.96691836e-06, 8.91923810e-06],
        [5.22719984e-06, 5.28792452e-06, 5.35071740e-06, ...,
         9.64615174e-06, 9.68001405e-06, 9.71249938e-06],
        [7.68844257e-06, 

In [91]:
rescaled_data

array([[[-1.57786590e-01, -1.39163747e-01, -1.21054572e-01, ...,
          1.23130149e-01,  1.14623107e-01,  1.05619729e-01],
        [-2.76209400e-01, -2.85319588e-01, -2.94684656e-01, ...,
         -2.04045934e+00, -2.01913759e+00, -1.99675968e+00],
        [ 9.15840779e-02,  9.53304710e-02,  9.87595422e-02, ...,
         -7.70256474e-02, -6.99058536e-02, -6.24311604e-02],
        ...,
        [-4.35821000e-02, -2.11303182e-02,  9.74899762e-04, ...,
          1.32120374e+00,  1.30796103e+00,  1.29390688e+00],
        [ 7.80726662e-02,  9.13117683e-02,  1.04432740e-01, ...,
          1.22255890e+00,  1.21842996e+00,  1.21355954e+00],
        [ 5.54203767e-01,  5.61491939e-01,  5.68686150e-01, ...,
          1.01649197e+00,  1.00826875e+00,  9.99539255e-01]],

       [[ 1.14319951e+00,  1.14546622e+00,  1.14685326e+00, ...,
         -1.90177859e+00, -1.90374953e+00, -1.90538260e+00],
        [-7.04356004e-01, -6.93456621e-01, -6.82372878e-01, ...,
          8.09486598e-02,  1.01335514e